In [8]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

print(f"TensorFlow Version: {tf.__version__}")


TensorFlow Version: 2.15.0


In [11]:
DATA_DIR = 'dataset_split'  # Folder containing 'train' and 'val'
IMG_SIZE = (224, 224) # Standard size for MobileNetV2
BATCH_SIZE = 32
EPOCHS = 20           # We need more than 1 epoch!

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255,         # Normalize pixel values (0-1)
    rotation_range=20,      # Rotate head slightly
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,         # Simulate distance changes
    horizontal_flip=True,   # Left eye vs Right eye
    fill_mode='nearest'
)

In [13]:
val_datagen = ImageDataGenerator(rescale=1./255)

print("Loading Training Data:")
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',     # Binary classification (0 vs 1)
    shuffle=True
)

Loading Training Data:
Found 37880 images belonging to 2 classes.


In [14]:
print("Loading Validation Data:")
val_generator = val_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


Loading Validation Data:
Found 15030 images belonging to 2 classes.


In [15]:
# --- CRITICAL CHECK ---
# Ensure 'Closed' is 0 and 'Open' is 1
print("\nClass Mappings:", train_generator.class_indices)
# Expectation: {'Closed': 0, 'Open': 1}



Class Mappings: {'Drowsy': 0, 'Non Drowsy': 1}


In [16]:

# Cell 3: Building the Model (Transfer Learning)

# 1. Download base model (MobileNetV2) without the top classification layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 2. Freeze the base model (so we don't destroy pre-trained patterns)
base_model.trainable = False


In [17]:
# 3. Add our custom layers on top
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)       # Condense features
x = Dense(128, activation='relu')(x)  # Learn complex patterns
x = Dropout(0.5)(x)                   # Prevent overfitting
outputs = Dense(1, activation='sigmoid')(x) # Output: 0.0 to 1.0

model = Model(inputs, outputs)

In [18]:
# 4. Compile
model.compile(optimizer=Adam(learning_rate=0.0001), # Low learning rate for fine-tuning
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129   

In [10]:
callbacks = [
    # Stop if validation loss doesn't improve for 5 epochs
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    # Save the best model automatically
    ModelCheckpoint('driver_drowsiness_final_model.h5', monitor='val_accuracy', save_best_only=True)
]

print("🚀 Starting Training...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks
)

🚀 Starting Training...
Epoch 1/20


1184/1184 [==============================] - ETA: 0s - loss: 0.3180 - accuracy: 0.8711

c:\Users\Vishal\OneDrive\IdeaProjects\driver drowsiness\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1184/1184 [==============================] - 3368s 3s/step - loss: 0.3180 - accuracy: 0.8711 - val_loss: 0.1275 - val_accuracy: 0.9811
Epoch 2/20
1184/1184 [==============================] - 2731s 2s/step - loss: 0.1203 - accuracy: 0.9659 - val_loss: 0.0669 - val_accuracy: 0.9891
Epoch 3/20
1184/1184 [==============================] - 2319s 2s/step - loss: 0.0719 - accuracy: 0.9813 - val_loss: 0.0485 - val_accuracy: 0.9907
Epoch 4/20
1184/1184 [==============================] - 2309s 2s/step - loss: 0.0500 - accuracy: 0.9874 - val_loss: 0.0291 - val_accuracy: 0.9941
Epoch 5/20
1184/1184 [==============================] - 2011s 2s/step - loss: 0.0376 - accuracy: 0.9903 - val_loss: 0.0204 - val_accuracy: 0.9953
Epoch 6/20
1184/1184 [==============================] - 1996s 2s/step - loss: 0.0285 - accuracy: 0.9930 - val_loss: 0.0156 - val_accuracy: 0.9951
Epoch 7/20
1184/1184 [==============================] - 2015s 2s/step - loss: 0.0243 - accuracy: 0.9932 - val_loss: 0.0129 - val_accura

In [19]:
model.save("driver_drowsiness_final_model.keras")
print("Model saved successfully!")
loaded = load_model("driver_drowsiness_final_model.keras")

loaded.summary()

Model saved successfully!


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129   

In [20]:
print("✅ Training Complete")

# Cell 5: Visualization
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Acc')
plt.plot(val_acc, label='Validation Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.show()


✅ Training Complete


NameError: name 'history' is not defined

In [21]:

# Cell 6: Test on a Single Image (Sanity Check)
# This mimics what the backend does
from keras.preprocessing import image

def predict_local_image(img_path ="WIN_20251130_15_26_10_Pro.jpg" ):
    if not os.path.exists(img_path):
        return
    
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    
    # CRITICAL: Normalize exactly like training (1./255)
    img_array /= 255.0
    
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array, verbose=0)
    val = prediction[0][0]
    
    print(f"\nImage: {img_path}")
    print(f"Raw Value: {val:.4f}")

    
    # Logic: 0=Closed, 1=Open
    if val < 0.5:
        print("Prediction: 😴 DROWSY (Closed)")
    else:
        print("Prediction: 👀 ALERT (Open)")

# You can upload test images to the notebook folder and run this
# predict_local_image('test_open.jpg')
# predict_local_image('test_closed.jpg')

In [22]:
predict_local_image("WIN_20251130_15_26_10_Pro.jpg")




Image: WIN_20251130_15_26_10_Pro.jpg
Raw Value: 0.6437
Prediction: 👀 ALERT (Open)
